In [2]:
DATA_NAME = 'batch0approx1e5-agriVision-fourier-red' # size-name-transform-channel

import git
from pathlib import Path
import os

ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
path_dict = dict(zip(['size', 'name', 'transform', 'channel'], DATA_NAME.split("-")))
CWD = os.path.join(ROOT_DIR, os.path.join("results", "case-studies", path_dict['name'], path_dict['transform'], path_dict['size'], path_dict['channel']))
assert CWD == os.getcwd()
Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "groupCDFs")).mkdir(exist_ok=True)

GROUP = 'layer' if path_dict['transform'] == 'wavelet' else ('band' if path_dict['transform'] == 'fourier' else 'error')
RERUN = False
CWD

'c:\\Users\\yashd\\Desktop\\hierarchical-bayesian-model-validation\\results\\case-studies\\agriVision\\fourier\\batch0approx1e5\\red'

In [4]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [ ]:
group_cdf_df_dict_path = Path(os.path.join(CWD, "cache", "group_cdf_df_dict_full.pickle"))
group_cdf_df_dict = pd.read_pickle(group_cdf_df_dict_path)

In [49]:
rEtaKsstats_dict = dict()
master_df = pd.read_csv(os.path.join(CWD, "CSVs", "master_df.csv"), index_col = GROUP)
master_df

,obs_var,var_lower,var_upper,obs_kurt,kurt_lower,kurt_upper,total_samples,initial_r,initial_eta,kstest_stat_initial,...,param_gaussian,kstest_stat_gaussian,kstest_pval_gaussian,param_laplace,kstest_stat_laplace,kstest_pval_laplace,param_t,kstest_stat_t,kstest_pval_t,kstest_pval_gengamma
band,,,,,,,,,,,,,,,,,,,,,
2,4.431448e+08,2.040596e+08,1.172902e+09,1951.210975,93.389987,3809.453405,45000.0,0.16,1.1,0.011800,...,6384.9600,0.044028,3.074840e-76,5694.050,0.018637,5.232690e-14,4473.4300,0.015720,4.328990e-10,8.772600e-02
5,2.425320e+08,1.167651e+08,6.376783e+08,1732.740712,102.459843,3959.468650,117000.0,0.17,1.5,0.011201,...,4468.0500,0.041173,8.895000e-173,3981.760,0.016246,2.978560e-27,3137.8700,0.011878,9.114010e-15,1.917900e-06
8,7.509878e+07,4.135542e+07,1.652329e+08,997.550338,76.816628,2699.581213,189000.0,0.17,1.1,0.006731,...,3071.4600,0.037180,1.935400e-227,2724.410,0.012882,1.127140e-27,2168.5900,0.009419,5.410980e-15,5.474200e-04
11,6.680555e+07,1.909183e+07,4.286984e+08,13976.007954,62.877469,8349.733185,495000.0,0.18,1.4,0.006411,...,2326.5400,0.035271,0.000000e+00,2054.270,0.010895,1.823320e-51,1646.1000,0.007017,1.337840e-21,7.975400e-07
14,1.964670e+07,9.259367e+06,7.427124e+07,4449.842883,48.653387,6157.999254,1134000.0,0.19,1.7,0.006926,...,1744.4800,0.034096,0.000000e+00,1540.840,0.009953,5.278280e-98,1238.7800,0.006253,6.126840e-39,5.208300e-19
17,9.104094e+06,4.527121e+06,3.404868e+07,4455.632700,42.402900,6093.362016,2574000.0,0.19,1.4,0.008478,...,1243.7800,0.034877,0.000000e+00,1098.810,0.009883,8.733880e-219,883.3900,0.007473,2.670040e-125,1.002900e-23
20,4.104360e+06,2.023371e+06,1.352017e+07,3024.862169,61.902663,4879.518751,5967000.0,0.19,0.9,0.005827,...,768.4440,0.037194,0.000000e+00,677.348,0.011672,0.000000e+00,545.3750,0.008923,0.000000e+00,3.431000e-51
23,1.838956e+06,6.761338e+05,1.053351e+07,11453.019888,71.847644,7815.204603,13887000.0,0.21,1.1,0.010091,...,418.3490,0.037800,0.000000e+00,369.150,0.013563,0.000000e+00,295.9530,0.007941,0.000000e+00,0.000000e+00
26,1.165550e+06,2.007962e+05,1.103744e+07,27543.829429,79.598564,9199.390263,31968000.0,0.20,0.2,0.007970,...,202.3580,0.044217,0.000000e+00,179.352,0.018980,0.000000e+00,142.5520,0.014343,0.000000e+00,0.000000e+00


In [57]:
for group in master_df.index:
    temp_df = group_cdf_df_dict[group].sort_values(['r', 'eta'])
    rEtaKsstats_dict[group] = [temp_df['r'], temp_df['eta'], temp_df['ksstat']]
pd.to_pickle(rEtaKsstats_dict, os.path.join(CWD, "cache", "test_rEtaKsstats_dict.pickle"))

In [60]:
rEtaKsstats_dict = pd.read_pickle(os.path.join(CWD, "cache", "test_rEtaKsstats_dict.pickle"))
rEtaKsstats_dict.keys()

dict_keys([2, 5, 8, 11, 14, 17, 20, 23, 26, 29])

In [ ]:
def add_tests_to_df(cdfs_df, group, var_kurt_df, ksstats):
    cdfs_df['pass_var'] = (cdfs_df['variance'] > var_kurt_df.loc[group, 'var_lower']) & (cdfs_df['variance'] < var_kurt_df.loc[group, 'var_upper'])
    cdfs_df['pass_kurt'] = (cdfs_df['kurtosis'] > var_kurt_df.loc[group, 'kurt_lower']) & (cdfs_df['kurtosis'] < var_kurt_df.loc[group,'kurt_upper'])
    cdfs_df['ksstat'] = ksstats
    cutoff = stats.kstwo(n=var_kurt_df.loc[group, 'total_samples']).isf(0.05)
    cdfs_df['pass_kstest'] = cdfs_df['ksstat'].apply(lambda x: True if x < cutoff else False)
    return cdfs_df

In [ ]:
group_cdf_df_dict = dict()
for group in master_df.index:
    group_cdf_df = pd.DataFrame(dict(zip(['r', 'eta', 'ksstat'], rEtaKsstats_dict[group])))
    group_cdf_df['variance'] = np.nan_to_num(group_cdf_df.apply(lambda row : variance_prior(row.loc['r'], row.loc['eta']), axis = 1))
    group_cdf_df['kurtosis'] = group_cdf_df.apply(lambda row : kurtosis_prior(row.loc['r'], row.loc['eta']), axis = 1)
    group_cdf_df = add_tests_to_df(cdfs_df=group_cdf_df, group=group, var_kurt_df=master_df, ksstats=group_cdf_df['ksstat']).sort_values(['r', 'eta'])
    group_cdf_df_dict[group] = group_cdf_df

c:\Users\yashd\Desktop\hierarchical-bayesian-model-validation\utilities\testing.py:818: RuntimeWarning: invalid value encountered in scalar divide
  var_prior = scale * scipy.special.gamma(beta + 1/r)/scipy.special.gamma(beta)
c:\Users\yashd\Desktop\hierarchical-bayesian-model-validation\utilities\testing.py:823: RuntimeWarning: overflow encountered in scalar multiply
  kurtosis = 3*scipy.special.gamma(beta + 2/r)*scipy.special.gamma(beta)/scipy.special.gamma(beta+1/r)**2
c:\Users\yashd\Desktop\hierarchical-bayesian-model-validation\utilities\testing.py:823: RuntimeWarning: overflow encountered in scalar power
  kurtosis = 3*scipy.special.gamma(beta + 2/r)*scipy.special.gamma(beta)/scipy.special.gamma(beta+1/r)**2
c:\Users\yashd\Desktop\hierarchical-bayesian-model-validation\utilities\testing.py:823: RuntimeWarning: invalid value encountered in scalar divide
  kurtosis = 3*scipy.special.gamma(beta + 2/r)*scipy.special.gamma(beta)/scipy.special.gamma(beta+1/r)**2
c:\Users\yashd\Desktop\

In [86]:
group_cdf_df_dict[2][(group_cdf_df_dict_reload[2] == group_cdf_df_dict[2][group_cdf_df_dict_reload[2].columns])['kurtosis'] == False]

,"(r,eta),cdf",r,eta,cdf,variance,kurtosis,pass_var,pass_kurt,ksstat,pass_kstest
9,"((0.02, -0.5), <scipy.interpolate._fitpack2.In...",0.02,-0.5,<scipy.interpolate._fitpack2.InterpolatedUniva...,1.534259e+93,NaN,False,False,0.500000,False
10,"((0.02, -0.4), <scipy.interpolate._fitpack2.In...",0.02,-0.4,<scipy.interpolate._fitpack2.InterpolatedUniva...,4.461468e+94,NaN,False,False,0.500000,False
11,"((0.02, -0.3), <scipy.interpolate._fitpack2.In...",0.02,-0.3,<scipy.interpolate._fitpack2.InterpolatedUniva...,1.041121e+96,NaN,False,False,0.500000,False
12,"((0.02, -0.2), <scipy.interpolate._fitpack2.In...",0.02,-0.2,<scipy.interpolate._fitpack2.InterpolatedUniva...,2.004588e+97,NaN,False,False,0.500000,False
13,"((0.02, -0.1), <scipy.interpolate._fitpack2.In...",0.02,-0.1,<scipy.interpolate._fitpack2.InterpolatedUniva...,3.257659e+98,NaN,False,False,0.500000,False
...,...,...,...,...,...,...,...,...,...,...
11842,"((0.9, 100), <scipy.interpolate._fitpack2.Inte...",0.90,100.0,<scipy.interpolate._fitpack2.InterpolatedUniva...,1.907593e+02,NaN,False,False,0.499596,False
13170,"((1.0, 97), <scipy.interpolate._fitpack2.Inter...",1.00,97.0,<scipy.interpolate._fitpack2.InterpolatedUniva...,9.850000e+01,NaN,False,False,0.500824,False
13171,"((1.0, 98), <scipy.interpolate._fitpack2.Inter...",1.00,98.0,<scipy.interpolate._fitpack2.InterpolatedUniva...,9.950000e+01,NaN,False,False,0.500808,False
13172,"((1.0, 99), <scipy.interpolate._fitpack2.Inter...",1.00,99.0,<scipy.interpolate._fitpack2.InterpolatedUniva...,1.005000e+02,NaN,False,False,0.500792,False


In [87]:
group_cdf_df_dict_reload[2].columns

Index(['r', 'eta', 'ksstat', 'variance', 'kurtosis', 'pass_var', 'pass_kurt',
       'pass_kstest'],
      dtype='object')

In [47]:
np.all(df['ksstat'] == temp['ksstat'])

True

In [ ]:

all_cdfs_df

c:\Users\yashd\Desktop\hierarchical-bayesian-model-validation\utilities\testing.py:818: RuntimeWarning: invalid value encountered in scalar divide
  var_prior = scale * scipy.special.gamma(beta + 1/r)/scipy.special.gamma(beta)
c:\Users\yashd\Desktop\hierarchical-bayesian-model-validation\utilities\testing.py:823: RuntimeWarning: overflow encountered in scalar multiply
  kurtosis = 3*scipy.special.gamma(beta + 2/r)*scipy.special.gamma(beta)/scipy.special.gamma(beta+1/r)**2
c:\Users\yashd\Desktop\hierarchical-bayesian-model-validation\utilities\testing.py:823: RuntimeWarning: overflow encountered in scalar power
  kurtosis = 3*scipy.special.gamma(beta + 2/r)*scipy.special.gamma(beta)/scipy.special.gamma(beta+1/r)**2
c:\Users\yashd\Desktop\hierarchical-bayesian-model-validation\utilities\testing.py:823: RuntimeWarning: invalid value encountered in scalar divide
  kurtosis = 3*scipy.special.gamma(beta + 2/r)*scipy.special.gamma(beta)/scipy.special.gamma(beta+1/r)**2


,r,eta,ksstat,variance,kurtosis
0,0.02,-1.4,0.485718,9.618487e+69,1.391529e+25
1,0.02,-1.3,0.499990,3.821735e+74,8.172634e+21
2,0.02,-1.2,0.500000,1.455488e+78,4.131785e+19
3,0.02,-1.1,0.500000,1.406735e+81,6.947275e+17
4,0.02,-1.0,0.500000,5.331443e+83,2.562075e+16
...,...,...,...,...,...
51878,100.00,16.0,0.504160,9.429738e-01,9.179352e-03
51879,100.00,17.0,0.504160,9.458591e-01,8.293833e-03
51880,100.00,18.0,0.504159,9.484740e-01,7.534577e-03
51881,100.00,19.0,0.504159,9.508561e-01,6.878576e-03
